In [1]:
import sys
import logging
import os
from collections import OrderedDict
import torch
import shutil
from torch.nn.parallel import DistributedDataParallel
from sklearn.model_selection import KFold
import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer, PeriodicCheckpointer
from detectron2.config import get_cfg
from detectron2.engine import default_argument_parser, default_setup, default_writers, launch
from detectron2.evaluation import (
    CityscapesInstanceEvaluator,
    CityscapesSemSegEvaluator,
    COCOEvaluator,
    COCOPanopticEvaluator,
    DatasetEvaluators,
    LVISEvaluator,
    PascalVOCDetectionEvaluator,
    SemSegEvaluator,
    inference_on_dataset,
    print_csv_format,
)
from detectron2.data import (
    DatasetCatalog,
    MetadataCatalog,
    build_detection_test_loader,
    build_detection_train_loader,
)
from detectron2.modeling import build_model
from detectron2.solver import build_lr_scheduler, build_optimizer
from detectron2.utils.events import EventStorage
import cv2
import pickle
from detectron2.structures import BoxMode
from detectron2.data import DatasetMapper
import detectron2.data.transforms as T
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
import numpy as np
from detectron2.utils.visualizer import Visualizer

In [2]:
def add_border(image, border_size, border_color):
    height, width = image.shape[:2]
    new_height = height + 2 * border_size
    new_width = width + 2 * border_size
    bordered_image = np.zeros((new_height, new_width, 3), dtype=np.uint8)
    bordered_image[border_size:height + border_size, border_size:width + border_size] = image
    cv2.rectangle(bordered_image, (0, 0), (new_width - 1, new_height - 1), border_color, border_size)
    return bordered_image
def get_evaluator(cfg, dataset_name, output_folder=None):
    """
    Create evaluator(s) for a given dataset.
    This uses the special metadata "evaluator_type" associated with each builtin dataset.
    For your own dataset, you can simply create an evaluator manually in your
    script and do not have to worry about the hacky if-else logic here.
    """
    if output_folder is None:
        output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
    evaluator_list = []
    evaluator_type = MetadataCatalog.get(dataset_name).evaluator_type
    if evaluator_type in ["sem_seg", "coco_panoptic_seg"]:
        evaluator_list.append(
            SemSegEvaluator(
                dataset_name,
                distributed=True,
                output_dir=output_folder,
            )
        )
    if evaluator_type in ["coco", "coco_panoptic_seg"]:
        evaluator_list.append(COCOEvaluator(dataset_name, output_dir=output_folder))
    if evaluator_type == "coco_panoptic_seg":
        evaluator_list.append(COCOPanopticEvaluator(dataset_name, output_folder))
    if evaluator_type == "cityscapes_instance":
        return CityscapesInstanceEvaluator(dataset_name)
    if evaluator_type == "cityscapes_sem_seg":
        return CityscapesSemSegEvaluator(dataset_name)
    if evaluator_type == "pascal_voc":
        return PascalVOCDetectionEvaluator(dataset_name)
    if evaluator_type == "lvis":
        return LVISEvaluator(dataset_name, cfg, True, output_folder)
    if len(evaluator_list) == 0:
        raise NotImplementedError(
            "no Evaluator for the dataset {} with the type {}".format(dataset_name, evaluator_type)
        )
    if len(evaluator_list) == 1:
        return evaluator_list[0]
    return DatasetEvaluators(evaluator_list)


In [3]:
class HubMapDataset:
    def __init__(self, image_dir, annotation_dir):
        self.image_dir = image_dir
        self.annotation_dir = annotation_dir
        self.image_file_list = sorted(os.listdir(self.image_dir))
        self.annotation_file_list = os.listdir(self.annotation_dir)
        
    def __len__(self):
        return len(self.image_file_list)
        
    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.image_file_list[idx])
        img_id = self.image_file_list[idx].split('.png')[0]
        annotation_path = os.path.join(self.annotation_dir, f'{img_id}.pkl')
        
        image = cv2.imread(image_path)
        height, width = image.shape[:2]
        
        record = {
            'file_name': image_path,
            'image_id': idx,
            'height': height,
            'width': width,
        }
        
        with open(annotation_path, 'rb') as f:
            orig_annots = pickle.load(f)
        
        objs = []
        for orig_annot in orig_annots:
            bbox = orig_annot['bbox']
            orig_annot['bbox'] = [bbox[0], bbox[1], bbox[0]+bbox[2], bbox[1]+bbox[3]]
            orig_annot['bbox_mode'] = BoxMode.XYXY_ABS
            objs.append(orig_annot)
            
        record['annotations'] = objs
        
        return record

In [4]:
CLASSES = ['blood_vessel']
def register_custom_dataset(dataset_name, image_dir, annotation_dir):
    DatasetCatalog.register(dataset_name, lambda: HubMapDataset(image_dir, annotation_dir))
    MetadataCatalog.get(dataset_name).set(thing_classes=CLASSES, evaluator_type="coco")

In [5]:
num_folds = 5
config_file = '/home/ec2-user/hubmap-hacking-the-human-vasculature/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml'
base_model_path = '/home/ec2-user/hubmap-hacking-the-human-vasculature/project_detectron2/output/inference'
base_dataset_path = '/home/ec2-user/hubmap-hacking-the-human-vasculature/dataset1_files'
base_dataset_name = 'hubmap-dataset1'
num_machines = 1
num_gpus = 1
machine_rank = 0
port = 2**15 + 2**14 + hash(os.getuid() if sys.platform != "win32" else 1) % 2**14
dist_url = "tcp://127.0.0.1:{}".format(port)
opts = []

In [6]:
register_custom_dataset(f'{base_dataset_name}-train-fold-0', f'{base_dataset_path}/all_dataset1_imgs_merged_train_0', f'{base_dataset_path}/all_dataset1_annotations_merged_train_0')
register_custom_dataset(f'{base_dataset_name}-validation-fold-0', f'{base_dataset_path}/all_dataset1_imgs_merged_validation_0', f'{base_dataset_path}/all_dataset1_annotations_merged_validation_0')

In [7]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file('COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml'))
cfg.MODEL.WEIGHTS = f'{base_model_path}/best_model_fold_0_current_best_2.pth'
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.5
cfg.MODEL.DEVICE = 'cpu'
predictor = DefaultPredictor(cfg)

In [ ]:
import time
# score_thresholds = [x for x in np.arange(0, 1, 0.1)]
# for score_thresh in score_thresholds:
start_time = time.time()
# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = float(score_thresh) if score_thresh>0 else 0.001
predictor = DefaultPredictor(cfg)
evaluator = get_evaluator(
    cfg, f'{base_dataset_name}-validation-fold-0', os.path.join(cfg.OUTPUT_DIR, "inference", f'{base_dataset_name}-validation-fold-0')
)
validation_data_loader = build_detection_test_loader(cfg, f'{base_dataset_name}-validation-fold-0')
print(f'=========Stats for score thresh: {cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST}==============')
print(inference_on_dataset(predictor.model, validation_data_loader, evaluator))
print(f'Time Taken: {(time.time()-start_time)/60} minutes')
print(f'=========Stats for score thresh: {cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST}==============')

=========Stats for score thresh: 0.05==============


/home/ec2-user/anaconda3/envs/detectron_env/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
# Get the metadata for your dataset
metadata = MetadataCatalog.get(f'{base_dataset_name}-validation-fold-0')

# Get the dataset dictionary
dataset_dicts = DatasetCatalog.get(f'{base_dataset_name}-validation-fold-0')

print(f'Dataset Dicsts length is: {len(dataset_dicts)}')
print(dataset_dicts[0])

In [ ]:
import matplotlib.pyplot as plt
import random
tgt_coco_dicts = [dataset_dicts[i] for i in range(50)]
# tgt_dict = None
# for d in dataset_dicts:
#     if 'e6d9bd78b840' in d['file_name']:
#         tgt_coco_dicts.append(d)
border_size = 2
border_color = (255, 0, 0)
score_thresholds = [x for x in np.arange(0, 1, 0.1)]

for i, d in enumerate(tgt_coco_dicts):
    # read the image with cv2
    img = cv2.imread(d["file_name"])
    test_data_loader = build_detection_test_loader(d, mapper=DatasetMapper(cfg, is_train=False))
#     print(d.keys())
#     print(d['annotations'])
    outputs = predictor(img)
    pred_masks = outputs['instances'].pred_masks
    pred_scores = outputs['instances'].scores
    print(d['file_name'])
    print(pred_scores.shape)
    print(pred_masks.shape)
    all_images = []
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1.0)
    vis = visualizer.draw_dataset_dict(d)
    img_with_ground_truth_overlay = vis.get_image()[:, :, ::-1]
    img_with_ground_truth_overlay = add_border(img_with_ground_truth_overlay, border_size, border_color)
    all_images.append(img_with_ground_truth_overlay)
    for score_thresh in score_thresholds:
        visualizer_pred = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1.0)
        visualizer_pred.overlay_instances(
            masks=pred_masks[pred_scores>score_thresh, :, :],
            alpha=0.5  # Adjust transparency if needed
            )
        img_with_mask_overlay = visualizer_pred.get_output().get_image()[:, :, ::-1]
        img_with_mask_overlay = add_border(img_with_mask_overlay, border_size, border_color)
        all_images.append(img_with_mask_overlay)
    num_imgs_in_slide = 3
    show_images_arr = []
    for i in range(len(all_images)):
        show_images_arr.append(all_images[i])
        if (i+1) % num_imgs_in_slide == 0:
            final_image = cv2.hconcat(show_images_arr)
            plt.figure(figsize=(50, 50))
            plt.imshow(final_image)
            plt.show()
            show_images_arr = []

In [ ]:
# Get the metadata for your dataset
metadata = MetadataCatalog.get(f'{base_dataset_name}-train-fold-0')

# Get the dataset dictionary
dataset_dicts = DatasetCatalog.get(f'{base_dataset_name}-train-fold-0')

print(f'Dataset Dicsts length is: {len(dataset_dicts)}')
print(dataset_dicts[0])

In [ ]:
import matplotlib.pyplot as plt
import random
tgt_coco_dicts = [dataset_dicts[i] for i in range(20)]
# tgt_dict = None
# for d in dataset_dicts:
#     if 'e6d9bd78b840' in d['file_name']:
#         tgt_coco_dicts.append(d)
border_size = 2
border_color = (255, 0, 0)
score_thresholds = [x for x in np.arange(0, 1, 0.1)]

for i, d in enumerate(tgt_coco_dicts):
    # read the image with cv2
    img = cv2.imread(d["file_name"])
#     print(d.keys())
#     print(d['annotations'])
    outputs = predictor(img)
    pred_masks = outputs['instances'].pred_masks
    pred_scores = outputs['instances'].scores
    print(d['file_name'])
    print(pred_scores.shape)
    print(pred_masks.shape)
    all_images = []
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1.0)
    vis = visualizer.draw_dataset_dict(d)
    img_with_ground_truth_overlay = vis.get_image()[:, :, ::-1]
    img_with_ground_truth_overlay = add_border(img_with_ground_truth_overlay, border_size, border_color)
    all_images.append(img_with_ground_truth_overlay)
    for score_thresh in score_thresholds:
        visualizer_pred = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1.0)
        visualizer_pred.overlay_instances(
            masks=pred_masks[pred_scores>score_thresh, :, :],
            alpha=0.5  # Adjust transparency if needed
            )
        img_with_mask_overlay = visualizer_pred.get_output().get_image()[:, :, ::-1]
        img_with_mask_overlay = add_border(img_with_mask_overlay, border_size, border_color)
        all_images.append(img_with_mask_overlay)
    num_imgs_in_slide = 3
    show_images_arr = []
    for i in range(len(all_images)):
        show_images_arr.append(all_images[i])
        if (i+1) % num_imgs_in_slide == 0:
            final_image = cv2.hconcat(show_images_arr)
            plt.figure(figsize=(50, 50))
            plt.imshow(final_image)
            plt.show()
            show_images_arr = []